## Preprocessing

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Looking at the data it appears that the SPECIAL_CONSIDERATIONS column isn't meaningful to the end result.
application_df.groupby(['APPLICATION_TYPE', 'SPECIAL_CONSIDERATIONS'])['IS_SUCCESSFUL'].count()

APPLICATION_TYPE  SPECIAL_CONSIDERATIONS
T10               N                           528
T12               N                            27
T13               N                            66
T14               N                             3
T15               N                             2
T17               N                             1
T19               N                          1065
T2                N                            16
T25               N                             3
T29               N                             2
T3                N                         27010
                  Y                            27
T4                N                          1542
T5                N                          1173
T6                N                          1216
T7                N                           725
T8                N                           737
T9                N                           156
Name: IS_SUCCESSFUL, dtype: int64

In [21]:
# Looking at the data it appears that the STATUS column isn't meaningful to the end result.
application_df.groupby(['APPLICATION_TYPE', 'STATUS'])['IS_SUCCESSFUL'].count()

APPLICATION_TYPE  STATUS
T10               1           528
T12               1            27
T13               1            66
T14               1             3
T15               1             2
T17               1             1
T19               1          1065
T2                1            16
T25               1             3
T29               1             2
T3                0             5
                  1         27032
T4                1          1542
T5                1          1173
T6                1          1216
T7                1           725
T8                1           737
T9                1           156
Name: IS_SUCCESSFUL, dtype: int64

In [22]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME', 'SPECIAL_CONSIDERATIONS' AND 'STATUS'.
application_df.drop(columns=['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS'], inplace = True)

In [23]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [24]:
# Create a cutoff function to apply to future cutoffs

def CutOff(cut_off_value, dataframe_column, dictionary_name):
    
    # Choose a cutoff value and create a list of application types to be replaced
    # use the variable name `application_types_to_replace`
    cutoff_value = cut_off_value
    dictionary_name = []

    # Get the value counts of each application type
    column_counts = dataframe_column.value_counts()

    # Create a list of application types to be replaced
    for type, count in column_counts.items():
        if count < cutoff_value:
            dictionary_name.append(type)

    # Replace in dataframe
    for each in dictionary_name:
        dataframe_column = dataframe_column.replace(each, "Other")

    # Check to make sure binning was successful
    return dataframe_column.value_counts()

In [25]:
# Income above 5M has such a small candidate pool it may not play a major factor in training the data. Set a filter level of 500.
application_df['INCOME_AMT'].value_counts()

CutOff(500, application_df['INCOME_AMT'], 'income_amoutns_to_replace')

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [26]:
print(application_df['APPLICATION_TYPE'].value_counts())

CutOff(150, application_df['APPLICATION_TYPE'], 'application_types_to_replace')

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [27]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
print(application_df['CLASSIFICATION'].value_counts())

CutOff(115, application_df['CLASSIFICATION'], 'classifications_to_replace')


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other      887
C7000      777
C1700      287
C4000      194
C5000      116
Name: CLASSIFICATION, dtype: int64

In [28]:
CutOff(115, application_df['CLASSIFICATION'], 'classifications_to_replace')

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other      887
C7000      777
C1700      287
C4000      194
C5000      116
Name: CLASSIFICATION, dtype: int64

In [29]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [30]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=39)

In [31]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [32]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Number of input features is equal to the shape of a single element of X_train_scaled
input_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Add a third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 50)                5700      
                                                                 
 dense_5 (Dense)             (None, 50)                2550      
                                                                 
 dense_6 (Dense)             (None, 50)                2550      
                                                                 
 dense_7 (Dense)             (None, 25)                1275      
                                                                 
 dense_8 (Dense)             (None, 1)                 26        
                                                                 
Total params: 12,101
Trainable params: 12,101
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5728 - accuracy: 0.7198
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7314
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7330
Epoch 4/50
766/804 [===========================>..] - ETA: 0s - loss: 0.5452 - accuracy: 0.7341

KeyboardInterrupt: 

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5670 - accuracy: 0.7263 - 688ms/epoch - 3ms/step
Loss: 0.5670236349105835, Accuracy: 0.7262973785400391


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityOptimized_3.h5")